In [1]:
from langchain.utilities import SQLDatabase  # Connect to the mysql database
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
load_dotenv()
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.messages import HumanMessage,AnyMessage,AIMessage,ToolMessage,SystemMessage
from langchain.tools import tool
from langchain.agents import AgentExecutor , create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from langchain_core.runnables import RunnableLambda,RunnableParallel,RunnablePassthrough,RunnableSequence
from langchain_core.messages import trim_messages
from langchain_groq import ChatGroq


# 1. Connect to Mysql Database

In [2]:
# Credentials such as username,password,database name etc 
username = 'readonly_user'
password = 'prabal9869'
host = '127.0.0.1'
db_name = 'nepa_wholesale'

mysql_uri = f"mysql+pymysql://{username}:{password}@{host}/{db_name}"

db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info=2)

In [3]:
# db.run("Drop table cigars_category;")

In [4]:
print(db.dialect)
print(db.get_usable_table_names()) # to get the name of tables
# print(db.get_table_info()) # to get the schema of the tables

mysql
['cigars_category', 'cigrattes_category', 'disposable_category', 'ecigs_vape_category', 'other_category', 'tobaccos_category']


# 2. Initialize the llm and embedding-model

In [5]:
# llm = ChatOpenAI(model= "gpt-4o mini",temperature = 0.3,max_tokens = 20)
llm = ChatGroq(model = "Llama-3.3-70b-Versatile",temperature=0.3,max_tokens = 300)

# llm = ChatOpenAI(model_name = 'gpt-3.5-turbo',temperature=0.2, max_tokens = 400)

In [6]:
# llm.invoke("Hi")

In [7]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\Prabal Kuinkel\Desktop\NepaWholesale\SQL_Agent\sql_agent_nepa\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. Creating fewshot examples

In [8]:
examples = [
    {
        'question': "How many products of fronto are there in x ?",
        'query': "SELECT COUNT(*) FROM x WHERE Brand LIKE '%FRONTO%';"
    },
    {
        'question': "How many products are there in x with flavor mint",
        'query': "SELECT COUNT(Product_ID) FROM x WHERE Flavor LIKE '%mint%';"
    },
    {
        'question': "How many products are there in x having nicotine less than 5%. list them ?",
        'query': "SELECT Product_ID, Display_Name, Nicotine_strength FROM x WHERE CAST(REPLACE(Nicotine_strength, '%', '') AS DECIMAL(3,2)) < 5 LIMIT 5;"
    },
    {
        'question': "Which flavors of x have 'Raspberry' in them?",
        'query': "SELECT Flavor FROM x WHERE Flavor LIKE '%Raspberry%' LIMIT 5;"
    },
    {
        'question': "List x products with nicotine strength of 2%",
        'query': "SELECT Display_Name, Nicotine_strength FROM x WHERE Nicotine_strength LIKE '%2%%' LIMIT 5;"
    },
    {
        'question': "How many different x flavors do you offer?",
        'query': "SELECT COUNT(DISTINCT Flavor) FROM x;"
    }
]


# 4. Creating the vectorstore

In [9]:
vectorstore = Chroma(
    embedding_function = embedding_model, # Embedding model
    collection_name = "example_collection", # Table name in vectorstore
    persist_directory="./chroma_db" 
)

In [10]:
# Addind the examples in vectorstore (Only questions as text and entire 'question and query' in metadata)
vectorstore.add_texts([ex['question'] for ex in examples],metadatas=examples)

['2c47764b-5fb3-4676-8ece-b318d19e0122',
 '818dbf08-a232-4b19-881c-17e9b6032d60',
 '38e12f8b-942f-491e-b923-432048404640',
 '038a51c5-c5fe-40bb-a635-c3ad59327b7a',
 'c4ebbfc7-f7eb-4f78-9a6e-aacb3965c7d2',
 '8918f447-5a3a-47b9-a2c1-a4c892db0915']

# 5. Defining the examples selector from vectorstore

In [11]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings = embedding_model,
    k=1 , 
    input_keys = ['question'],   # Which key to use for similarity search
    example_keys=['question','query'], # Which keys to return
    vectorstore_cls=Chroma,
)

# 6. Build dynamic prompt with : prompt + examples 

In [12]:
def build_dynamic_prefix(user_question: str) -> str:
    # print(messages[-1].content)
    # print(user_question)
    # user_question = messages
    # print(f"***** Selectiong Dynamic example using this question***** {user_question}")
    # user_question = 
    selected = example_selector.select_examples({"question": user_question})
    formatted_examples = "\n".join([
        f"Human: {ex['question']}\nAI:\nSQLQuery: {ex['query']}" for ex in selected
    ])
    return f"""User Question to answer :  {user_question}

Refer to the below given most similar examples to answer the above user question . 

Examples:
{formatted_examples}
...

Now begin.
"""
# dynamic_prefix = build_dynamic_prefix("what is my income of last year")

In [13]:
print(build_dynamic_prefix("list me the disposables products that have nicotine strength between 0% to 5% ."))

User Question to answer :  list me the disposables products that have nicotine strength between 0% to 5% .

Refer to the below given most similar examples to answer the above user question . 

Examples:
Human: List x products with nicotine strength of 2%
AI:
SQLQuery: SELECT Display_Name, Nicotine_strength FROM x WHERE Nicotine_strength LIKE '%2%%' LIMIT 5;
...

Now begin.



# 7. Using Create_SQL_agent with agent_type "Openai_Functions"

### Note : Only works with 'OPEN-AI MODELS"

In [14]:

# tool to get the list of tables from database
@tool
def list_tables(_: str = "") -> str:
    """Use this tool to list all table names that exist in the connected MySQL database. Pass an empty string to get the list of available tables
    This is helpful when you need to know what tables are available before writing a SQL query."""
    return str(db.get_table_names())


# tool to get schema of table to be used
@tool
def describe_table(table_name: str) -> str:
    """Use this tool to get the schema (column names and types) of a specific table.Input should be the name of the table as a string. 
    This is useful to understand what data is stored in the table before writing queries."""
    return str(db.get_table_info([table_name]))

# tool to execute the sql query in database
@tool
def run_sql_query(query: str) -> str:
    """Use this tool to run a raw SQL query on the database and return the result.
    Input should be a complete and valid SQL SELECT query as a string. 
    Use this when you already know which table and columns to query."""
    return str(db.run(query))





In [15]:
print(run_sql_query.name)  #to get the name of the tool
print(run_sql_query.description) #to get the description of the tool
print(run_sql_query.args) # to get the input arguments to the tools 
print(run_sql_query.args_schema.model_json_schema()) # to get the schmea of the tool 

run_sql_query
Use this tool to run a raw SQL query on the database and return the result.
    Input should be a complete and valid SQL SELECT query as a string. 
    Use this when you already know which table and columns to query.
{'query': {'title': 'Query', 'type': 'string'}}
{'description': 'Use this tool to run a raw SQL query on the database and return the result.\nInput should be a complete and valid SQL SELECT query as a string. \nUse this when you already know which table and columns to query.', 'properties': {'query': {'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'run_sql_query', 'type': 'object'}


In [16]:
chat_history = []

In [17]:
system_msg = """
You are a helpful assistant that interacts with a MySQL database using tools. Your task is to create syntactically correct MySQL SELECT queries, execute them using tools, and return a well-formatted, natural language response.
Rules:

1. Use tools in this order: list_tables ,describe_table ,run_sql_query.
2. Only handle simple greetings like "Hi" or "Hello" without using tools.
3. Strictly For SQL queries, use only SELECT statements. Do not execute queries queries such as INSERT, UPDATE, DELETE,ALTER ,DROP etc that modifies the schema of the database etc.
4. Do not show SQL queries in the final user response.
5. Retrieve a maximum of 5 results using LIMIT unless the user specifies otherwise.
6. Never query all columns; retrieve only the necessary ones.
7. Do not hallucinate answers. If data is not found, respond with: "Sorry I am unable to answer your question."
8. Always handle follow-up questions based on previous chat history and use tools if needed.

Available tools:

1. list_tables – Lists all tables.
2. describe_table – Describes schema of a table.
3. run_sql_query – Executes a raw SQL SELECT query.

Use the tools smartly to identify the right table, understand the schema, generate the right query, and answer concisely as You are a Customer supporter Agent.
"""

In [18]:
# Trim messages to keep last 2 conversation
def conversational_window_memory(messages):
    selected_msg = trim_messages(
    messages,
    token_counter=len,  
    max_tokens=2,  
    strategy="last",
    
    start_on="human",

    include_system=True,
    allow_partial=False,

    )
    return selected_msg

In [19]:
tools = [list_tables,describe_table,run_sql_query]

def initialize_agent(prompt):
    agent = create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)

    agent_executor2 = AgentExecutor(agent=agent, tools=tools, verbose=True,return_intermediate_steps=True)
    return agent , agent_executor2

In [20]:
## Assembling all the components

user_question = input("Enter the query: ")
if user_question:
    
    dynamic_prefix = build_dynamic_prefix(user_question) # User question + 1 most similar examples from vectorstore

    prompt = ChatPromptTemplate.from_messages([          # Prompt to agent that conist of : system prompt + chat_history + user_question and dynamic example + Empty agent scratchpad
    ("system", system_msg),

    MessagesPlaceholder(variable_name = "chat_history"),

    ("human",dynamic_prefix),
    MessagesPlaceholder(variable_name="agent_scratchpad")  #
])
    
    agent,agent_executor = initialize_agent(prompt)  # Initialize the agent and agent executor with the prompt

    response = agent_executor.invoke({               # Invoke the agent with empty "" input and chat_history
    "chat_history" : chat_history 
    })
    print(f"User Question: {user_question}")
    print(f"Agent: {response}")

    chat_history.append(HumanMessage(content=user_question))        # Append the user question to chat_history

    chat_history.append(AIMessage(response['output']))                #Append the agent's final response to chat_history

    chat_history = conversational_window_memory(chat_history)          # Trim to keep only 2 msg (equivalent to 1 conversation between human and agent)
    




> Entering new AgentExecutor chain...

Invoking: `list_tables` with `{}`


['cigars_category', 'cigrattes_category', 'disposable_category', 'ecigs_vape_category', 'other_category', 'tobaccos_category']

C:\Users\Prabal Kuinkel\AppData\Local\Temp\ipykernel_14832\3841982296.py:6: LangChainDeprecationWarning: The method `SQLDatabase.get_table_names` was deprecated in langchain-community 0.0.1 and will be removed in 1.0. Use :meth:`~get_usable_table_names` instead.
  return str(db.get_table_names())



Invoking: `describe_table` with `{'table_name': 'cigars_category'}`



CREATE TABLE cigars_category (
	`Product_ID` INTEGER NOT NULL, 
	`Display_Name` VARCHAR(225), 
	`Barcode` VARCHAR(50), 
	`Product_Sub_Category` VARCHAR(80), 
	`Product_Category` VARCHAR(80), 
	`Brand` VARCHAR(80), 
	`Flavor` VARCHAR(80), 
	`Packet_count` VARCHAR(30), 
	`Other_Features` VARCHAR(200), 
	PRIMARY KEY (`Product_ID`)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
2 rows from cigars_category table:
Product_ID	Display_Name	Barcode	Product_Sub_Category	Product_Category	Brand	Flavor	Packet_count	Other_Features
5	AL CAPONE 10-10PK (Sweet Filter)	7.18E+11	AL CAPONE	CIGARS	AL CAPONE	Sweet Filter	10-10PK	
693	AL CAPONE 60-2PK (Sweet Filter Tower)	7.18E+11	AL CAPONE	CIGARS	AL CAPONE	Sweet Filter Tower	60-2PK	
*/
Invoking: `run_sql_query` with `{'query': 'SELECT COUNT(DISTINCT Flavor) FROM cigars_category'}`


[(664,)]We have 664 different flavors of cigars.

> Finished chain.
User Question:

In [21]:
print(f"User Question: {user_question}")
print(f"Agent: {response['output']}")
print(chat_history)

User Question: how many flavors do you have in cigars .
Agent: We have 664 different flavors of cigars.
[HumanMessage(content='how many flavors do you have in cigars .', additional_kwargs={}, response_metadata={}), AIMessage(content='We have 664 different flavors of cigars.', additional_kwargs={}, response_metadata={})]


In [22]:
response

{'chat_history': [HumanMessage(content='how many flavors do you have in cigars .', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We have 664 different flavors of cigars.', additional_kwargs={}, response_metadata={})],
 'output': 'We have 664 different flavors of cigars.',
 'intermediate_steps': [(ToolAgentAction(tool='list_tables', tool_input={}, log='\nInvoking: `list_tables` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_35vj', 'function': {'arguments': '{}', 'name': 'list_tables'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'Llama-3.3-70b-Versatile', 'system_fingerprint': 'fp_2ddfbb0da0'}, id='run--80e28f6f-928b-4605-ac63-49f28f6ddb35', tool_calls=[{'name': 'list_tables', 'args': {}, 'id': 'call_35vj', 'type': 'tool_call'}], usage_metadata={'input_tokens': 893, 'output_tokens': 41, 'total_tokens': 934}, tool_call_chunks=[{'name': 'list_tables', 'args'

In [23]:
response['intermediate_steps'][2][0].tool_input['query']

'SELECT COUNT(DISTINCT Flavor) FROM cigars_category'

In [24]:
sql_query = None

for step in response.get("intermediate_steps", []):
    action, _ = step
    if hasattr(action, "tool") and action.tool == "run_sql_query":
        sql_query = action.tool_input.get("query")
        break

if sql_query:
    print(sql_query)
else:
    print("No SQL query found in intermediate steps.")

SELECT COUNT(DISTINCT Flavor) FROM cigars_category


In [27]:
for step in response.get("intermediate_steps", []):
        action, _ = step
        if hasattr(action, "tool") and action.tool == "run_sql_query":
            sql_query = action.tool_input.get("query")
            tool_call_id = action.tool_call_id
            break
print(sql_query)
print(tool_call_id)

SELECT COUNT(DISTINCT Flavor) FROM cigars_category
call_cdrt


### ---------------------------------------------------------------------------------------------------------------------------------